In [62]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler, StandardScaler, FunctionTransformer
from sklearn.metrics import plot_roc_curve, accuracy_score, confusion_matrix, mean_squared_error, roc_auc_score, r2_score                 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer

In [2]:
testv = pd.read_csv('../data/test_vals.csv')
trainl = pd.read_csv('../data/training_labels.csv')
trainv = pd.read_csv('../data/training_vals.csv')

In [3]:
trainl.status_group.value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [4]:
idequality = (trainl['id'] == trainv['id'] )

In [5]:
data = pd.merge(trainv, trainl) #merging into one dataset


In [6]:
print(set(data.columns).difference(set(trainv.columns))) # a sort of check to see if merge worked

{'status_group'}


In [7]:
# testohe = OneHotEncoder()
# dummy1 = pd.get_dummies(data['scheme_name'].dropna())
# dummy2 = pd.get_dummies(data['status_group'].dropna())
# testtree = DecisionTreeClassifier(max_depth=3)
# testtree.fit(dummy1, dummy2 )/
# testtree.score(data['scheme_name'], data['status_group'])

In [8]:
# pipe = Pipeline(steps=[
#     ('scaler', StandardScaler),
#     ('ohe', OneHotEncoder)
#     ('polynom', PolynomialFeatures)
# ])

In [9]:
data['waterpoint_type_group'].unique(), data['waterpoint_type'].unique()

(array(['communal standpipe', 'hand pump', 'other', 'improved spring',
        'cattle trough', 'dam'], dtype=object),
 array(['communal standpipe', 'communal standpipe multiple', 'hand pump',
        'other', 'improved spring', 'cattle trough', 'dam'], dtype=object))

In [10]:
data.info() #which to drop, which to encode, etc

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

TODO: dropnas on all, make copy df drop some columns like ['date_recorded', 'latitude', 'longitude', 'wpt_name', 'num_private', 'region_code', 'district_code, 'lga', 'ward', 'subvillage', 'public_meeting', 'recorded_by', 'extraction_type', 'extraction_type_group', 'management_group', 'payment_type', 'water_quality', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type']


^^ all but 'region' for location, then some unhelpful others such as ones without clear description

&nbsp;

get rid of zero val in construction_year 
change year into daterange from base year point

&nbsp;

check target as boolean vs multiclass vs maybe smoted functional but needs repair

&nbsp;

make narrative (stakeholder etc)

&nbsp;

dropnas, check for zeros, maybe convert bools, ordinal?, ohe, regression/normalize, model different versions (drop/dont scheme_name, drop/dont construction_year)

In [11]:
dropt_cols = ['id', 'date_recorded', 'latitude', 'longitude', 'wpt_name', 'num_private', 'region_code', 'district_code', 'lga', 'ward', 'subvillage', 'public_meeting', 'recorded_by', 'extraction_type', 'extraction_type_group', 'management_group', 'payment_type', 'water_quality', 'quantity_group', 'source_type', 'source_class', 'waterpoint_type']

len(dropt_cols)

22

In [12]:
data1 = data.drop(
    ['date_recorded', 'latitude', 'longitude', 'wpt_name', 
     'num_private', 'region_code', 'district_code', 'lga',
     'ward', 'subvillage', 'public_meeting', 'recorded_by', 
     'construction_year', 'extraction_type', 
     'extraction_type_group', 'management_group', 'payment_type',
     'water_quality', 'quantity_group', 'source_type',
     'source_class', 'waterpoint_type'],
    axis=1)

data1.dropna(inplace=True)

In [13]:
# # is it possible to make a train test split function?
# def ttsplit(X, y, n):
#     Xtrain, Xtest, ytrain, ytest = train_test_split(X,y)
# this was achieved in collab with Andrew, with a for loop iterating through a dictionary to name variables and run them through train_test_split

In [14]:
drop_cols_list = ['id','date_recorded', 'latitude', 'longitude', 'wpt_name',
                  'num_private', 'region_code', 'district_code', 'lga', 'ward',
                  'subvillage', 'public_meeting', 'recorded_by', 'extraction_type', 
                  'extraction_type_group', 'management_group', 'payment_type', 'water_quality',
                  'quantity_group', 'source_type', 'source_class', 'waterpoint_type']

df1 = data.drop(drop_cols_list, axis=1)
df2 = df1.dropna()
df3 = df2.drop(df2.loc[df2['construction_year']==0].index)
data_all = df3 
df4 = df1.drop('scheme_name', axis=1)
df5 = df4.dropna()
df6 = df5.drop(df5.loc[df5['construction_year']==0].index)
data_no_sname = df6
df7 = df1.drop('construction_year', axis=1)
df8 = df7.dropna()
data_no_constr = df8
df9 = df1.drop(['scheme_name', 'construction_year'], axis=1)
df10 = df9.dropna()
data_no_sname_no_constr = df10


In [15]:
df_list = {'data_all': data_all, 'data_no_sname': data_no_sname, 'data_no_constr': data_no_constr, 'data_no_sname_no_constr': data_no_sname_no_constr}
df_dic = {}

for key, value in df_list.items():
    X = value.drop('status_group', axis=1)
    y = value['status_group']
    z = key
    df_dic["X_train_" + z], df_dic["X_test_" + z], df_dic["y_train_" + z], df_dic["y_test_" + z] = train_test_split(X, y)

In [25]:
# def grab_numeric(df):
#     return df.select_dtypes(include=['float', 'int'])
# GrabNum = FunctionTransformer(grab_numeric)

def grab_numeric(df):
    df = pd.DataFrame(df)
    return df.select_dtypes(include=['float', 'int']) 

GrabNum = FunctionTransformer(grab_numeric)

subpipe_num = Pipeline(steps = [('grab_num', GrabNum), ('scaler', StandardScaler())])

def grab_categorical(df):
    df = pd.DataFrame(df)
    return df.select_dtypes(include=['object'])

GrabCat = FunctionTransformer(grab_categorical)

subpipe_cat = Pipeline(steps = [('grab_cat', GrabCat),
                                ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

col_trans = ColumnTransformer(transformers=[('subpipe_num', subpipe_num, X.columns),
                                            ('subpipe_cat', subpipe_cat, X.columns)]) 



In [39]:
logreg_pipe = Pipeline(steps=[('col_trans', col_trans), ('logreg', LogisticRegression(max_iter=1000))]) 

    "from sklearn.pipeline import Pipeline\n",
    "## Setting Up a Pipeline"
    "numTrans = Pipeline(steps=[\n",
    "catTrans = Pipeline(steps=[\n",
       "                                 Pipeline(steps=[('scaler', StandardScaler())]),\n",
       "                                 Pipeline(steps=[('ohe',\n",


In [35]:
%timeit logreg_pipe.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

42.7 s ± 1.83 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
!uptime

 14:59:40 up 38 days, 16:42,  0 users,  load average: 2.57, 3.96, 4.68


In [52]:
# logistic regression 
params = {}
params['logreg__penalty'] = ['l1', 'l2']
params['logreg__C'] = [1.0, 10.0, 100.0, 1000.0, 10000.0]
params['logreg__solver'] = ['saga', 'lbfgs']
# params['logreg__max_iter'] = []

grid = GridSearchCV(logreg_pipe, param_grid=params, cv=3, n_jobs=-7, verbose=3)

grid.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-7)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-7)]: Done  28 tasks      | elapsed: 65.7min
[Parallel(n_jobs=-7)]: Done  60 out of  60 | elapsed: 138.9min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('grab_num',
                                                                                          FunctionTransformer(func=<function grab_numeric at 0x7fd7ed96d5e0>)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extracti...
       'population', 'scheme_management', 'permit', 'extraction_type_class',
       'management', 'payment', 'quality_group', 'quantity', 'source',
    

In [58]:
forest_pipe = Pipeline(steps=[('col_trans', col_trans), ('forest', RandomForestClassifier())]) 

In [59]:
forest_pipe.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

Pipeline(steps=[('col_trans',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('grab_num',
                                                                   FunctionTransformer(func=<function grab_numeric at 0x7fd7ed96d5e0>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extraction_type_class',
       'management',...
                                                                   FunctionTransformer(func=<function grab_categorical at 0x7fd7ed96d670>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(

In [60]:
params0 = {}
params0['forest__n_estimators'] = [10, 100, 1000]
params0['forest__max_depth'] = [3, 5, 10]
params0['forest__min_samples_split'] = [200, 1000]
params0['forest__max_features'] = ['sqrt', 'log2']

# params['logreg__max_iter'] = []

grid0 = GridSearchCV(forest_pipe, param_grid=params0, cv=3, n_jobs=-2, verbose=3)

grid0.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-2)]: Done 108 out of 108 | elapsed:  3.1min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('grab_num',
                                                                                          FunctionTransformer(func=<function grab_numeric at 0x7fd7ed96d5e0>)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extracti...
       'population', 'scheme_management', 'permit', 'extraction_type_class',
       'management', 'payment', 'quality_group', 'quantity', 'source',
    

In [63]:
knn_pipe = Pipeline(steps=[('col_trans', col_trans), ('knn', KNeighborsClassifier())]) 

In [64]:
knn_pipe.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

Pipeline(steps=[('col_trans',
                 ColumnTransformer(transformers=[('subpipe_num',
                                                  Pipeline(steps=[('grab_num',
                                                                   FunctionTransformer(func=<function grab_numeric at 0x7fd7ed96d5e0>)),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extraction_type_class',
       'management',...
                                                                   FunctionTransformer(func=<function grab_categorical at 0x7fd7ed96d670>)),
                                                                  ('ohe',
                                                                   OneHotEncoder(

In [66]:
params1 = {}
params1['knn__n_neighbors'] = [3, 5, 7, 11]
params1['knn__weights'] = ['uniform', 'distance']
params1['knn__p'] = [1, 2]

grid1 = GridSearchCV(knn_pipe, param_grid=params1, cv=3, n_jobs=-1, verbose=3)

grid1.fit(df_dic['X_train_data_all'], df_dic['y_train_data_all'])

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:  7.2min finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(transformers=[('subpipe_num',
                                                                         Pipeline(steps=[('grab_num',
                                                                                          FunctionTransformer(func=<function grab_numeric at 0x7fd7ed96d5e0>)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
       'population', 'scheme_management', 'permit', 'extracti...
                                                                         Index(['amount_tsh', 'funder', 'gps_height', 'installer', 'basin', 'region',
  

In [68]:
grid.best_params_, grid0.best_params_, grid1.best_params_

({'logreg__C': 1.0, 'logreg__penalty': 'l2', 'logreg__solver': 'lbfgs'},
 {'forest__max_depth': 10,
  'forest__max_features': 'sqrt',
  'forest__min_samples_split': 200,
  'forest__n_estimators': 100},
 {'knn__n_neighbors': 11, 'knn__p': 1, 'knn__weights': 'distance'})